# DeepSpeed 가속기를 Amazon SageMaker 에서 사용하기

이 노트북은 DeepSpeed 를 SageMaker 에서 모델 학습을 하기 위해서 버트 모델을 네이버 영화 리뷰로 학습을 다루고 있습니다.
<br>
아래와 같이 3가지로 테스트를 할 수 있습니다.
- 현재의 노트북 인스턴스에서 모델 학습하기
    - 다음 섹션에서 진행 됩니다. --> ## 3. 로컬 (SageMaker Notebook Instance) 에서 실행 
- SageMaker 의 로컬 모드로 학습하기 ( 3. SageMaker Training 준비에서 USE_LOCAL_MODE = True 로 수정)
- SageMaker 의 클라우드 모드로 학습하기 ( 3. SageMaker Training 준비에서 USE_LOCAL_MODE = False 로 수정)

---

## 1. 사전 필수 사항

### 실습환경
- 이 노트북은 [SageMaker Notebook Instance](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html) 에서 테스트 완료 되었습니다.
    - 환경: ml.p3.2xlarge 및 생성시에 Additional configuration 클릭하고 200 GB EBS 추가 하였습니다. --> 참고: [Create a Notebook Instance](https://docs.aws.amazon.com/sagemaker/latest/dg/howitworks-create-ws.html)
### 분산 훈련
- 훈련(Training) job 수행 시 최소 ml.p3.2xlarge 이상의 훈련 인스턴스가 필요하며, 분산 훈련 핸즈온은 `ml.p3.16xlarge` 인스턴스를 권장합니다. 만약 인스턴스 사용에 제한이 걸려 있다면 [Request a service quota increase for SageMaker resources](https://docs.aws.amazon.com/sagemaker/latest/dg/regions-quotas.html#service-limit-increase-request-procedure)를 참조하여 인스턴스 제한을 해제해 주세요.
- instance_count 도 현재 1 로 세팅되어 있습니다. 2개의 ml.p3.2xlarge 및 ml.p4d.24xlarge 에서도 테스트 되었습니다.
### 환경 세팅
- 이 노트북을 실행하기 전에 [README.md](..setup/README.md) 를 참조하시고, 먼저 실행 해주세요.



### [중요] 설치된 패키지 확인
- 아래를 실행한면 `deepspeed 0.14.2` 등이 설치가 된 것을 확인할 수 있습니다.
- 만일 설치 패키지 결과가 보이지 않으면, 노트북 오른쪽 상단에 쥬피터 커널이 사전에 설치하신 커널로 선택 (예: deepspeed-py310) 이 되어 있는지 확인 해주세요.
- 그래도 문제가 해결이 되지 않으면, Web Brower 를 재로딩 해보세요.

In [1]:
print("## installed packages ")
! pip list | grep -E "datasets|transformers|fsspec|evaluate|deepspeed|s3fs|boto3|sagemaker|scikit-learn|torch"

## installed packages 
boto3                     1.34.101
datasets                  2.14.6
deepspeed                 0.14.2
evaluate                  0.4.0
fsspec                    2023.10.0
s3fs                      0.4.2
sagemaker                 2.219.0
scikit-learn              1.4.1.post1
torch                     2.1.0
transformers              4.30.2


## 2. 데이터 셋 준비
---

SageMaker 훈련을 위해 전처리된 데이터셋을 S3에 업로드합니다.

### SageMaker 사용 위한 역할, 리젼 등의 정보 얻기

In [2]:
import os
import sys
import logging
import boto3
import botocore
import sagemaker
import time
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

logging.info(f"sagemaker role arn: {role}")
logging.info(f"sagemaker bucket: {sess.default_bucket()}")
logging.info(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
[{2147392357.py:28} INFO - sagemaker role arn: arn:aws:iam::057716757052:role/workshop-sagemaker-kfp-role2
[{2147392357.py:29} INFO - sagemaker bucket: sagemaker-us-east-1-057716757052
[{2147392357.py:30} INFO - sagemaker session region: us-east-1


### 모델 선택 및 데이터 셋 정의
- 모델은 [bert-base-multilingual-cased](https://huggingface.co/google-bert/bert-base-multilingual-cased) 를 사용합니다. model_id 를 hyperparmeter 로 제공하여, 훈련 스크립트 (scripts/train_deepspeed.py) 에서 BertForSequenceClassification 를 통하여 모델을 로딩 합니다.
- 데이터 세트는 네이버 영화 리뷰 데이터 세트를 로딩 합니다.  [e9t/nsmc](https://huggingface.co/datasets/e9t/nsmc)
- 


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Define the model repo
model_id = 'bert-base-multilingual-cased'

# dataset used
dataset_name = 'nsmc'

# s3 key prefix for the data
s3_prefix = 'datasets/nsmc'

/home/ec2-user/anaconda3/envs/deepspeed-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 데이터 셋 로딩

In [4]:
# load dataset
train_dataset, eval_dataset = load_dataset(dataset_name, split=['train', 'test'])

num_samples_for_debug = 2000
train_dataset = train_dataset.shuffle(seed=42).select(range(num_samples_for_debug))
eval_dataset = eval_dataset.shuffle(seed=42).select(range(num_samples_for_debug))

logging.info(f" loaded train_dataset length is: {len(train_dataset)}")
logging.info(f" loaded eval_dataset length is: {len(eval_dataset)}")
logging.info(train_dataset[0])

Generating test split: 100%|██████████| 50000/50000 [00:01<00:00, 35297.61 examples/s]

[{3407595512.py:8} INFO -  loaded train_dataset length is: 2000
[{3407595512.py:9} INFO -  loaded eval_dataset length is: 2000
[{3407595512.py:10} INFO - {'id': '10020916', 'document': 'For Carl.칼 세이건으로 시작해서 칼 세이건으로 끝난다.', 'label': 1}


In [5]:
train_dataset, train_dataset[0]

(Dataset({
     features: ['id', 'document', 'label'],
     num_rows: 2000
 }),
 {'id': '10020916',
  'document': 'For Carl.칼 세이건으로 시작해서 칼 세이건으로 끝난다.',
  'label': 1})

### 데이터 셋의 인코딩
- 데이터 셋의 document 의 텍스트를 토큰나이저를 통하여 input_ids, attention_mask 로 변환 합니다.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch['document'], padding='max_length', max_length=128, truncation=True)

# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=['id', 'document'])
eval_dataset = eval_dataset.map(tokenize, batched=True, remove_columns=['id', 'document'])

# set format for pytorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

train_dataset = train_dataset.rename_column("label", "labels")
eval_dataset = eval_dataset.rename_column("label", "labels")

/home/ec2-user/anaconda3/envs/deepspeed-py310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 2000/2000 [00:00<00:00, 9253.69 examples/s]


In [7]:
train_dataset, train_dataset[0]

(Dataset({
     features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 2000
 }),
 {'labels': tensor(1),
  'input_ids': tensor([  101, 11399, 12225,   119,  9788,  9435, 10739, 71439, 11467,  9485,
          38709, 70146,  9788,  9435, 10739, 71439, 11467,  8977, 33305, 11903,
            119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0, 

### 데이터 셋을 Local 에 저장 

In [8]:
train_dir = 'train'
eval_dir = 'eval'
!rm -rf {train_dir} {eval_dir}

os.makedirs(train_dir, exist_ok=True)
os.makedirs(eval_dir, exist_ok=True) 

if not os.listdir(train_dir):
    train_dataset.save_to_disk(train_dir)
if not os.listdir(eval_dir):
    eval_dataset.save_to_disk(eval_dir)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saving the dataset (1/1 shards): 100%|██████████| 2000/2000 [00:00<00:00, 282978.28 examples/s]


In [9]:
# save train_dataset to s3
train_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/{train_dir}'
train_dataset.save_to_disk(train_input_path)

print("")
print("train_input_path:\n", train_input_path)
# save eval_dataset to s3
eval_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/{eval_dir}'
eval_dataset.save_to_disk(eval_input_path)
print("eval_input_path:\n", eval_input_path)

[{credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


/home/ec2-user/anaconda3/envs/deepspeed-py310/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
Saving the dataset (1/1 shards): 100%|██████████| 2000/2000 [00:00<00:00, 10399.63 examples/s]



train_input_path:
 s3://sagemaker-us-east-1-057716757052/datasets/nsmc/train


Saving the dataset (1/1 shards): 100%|██████████| 2000/2000 [00:00<00:00, 17113.08 examples/s]


eval_input_path:
 s3://sagemaker-us-east-1-057716757052/datasets/nsmc/eval


<br>

## 3. 로컬 (SageMaker Notebook Instance) 에서 실행 
- 개발 단계에서 코드의 생성, 간단한 실험 및 디버깅시에 주로 사용합니다.
---

- SageMaker에서 훈련을 수행하기 전에 먼저 로컬 개발 환경에서 모델 훈련 코드를 개발하고 디버깅해야 합니다. SageMaker 노트북 인스턴스에서 작업하는 경우 GPU가 탑재된 인스턴스(p-family)를 사용하셔야 합니다.
- [중요] 참고로 아래의 실행은 "로컬 환경 세팅" 이 정확하지 않으면, 에러가 자주 발생합니다. 
- 이후에 다른 버전 (예: Python, PyTorch ) 으로 업그레이드시에, 현재의 설정읠 참고 하시면서 업그레이드 하세요.

### DeepSpeed 로컬 실행

In [10]:
%%time

TRAIN_DEEPSPEED_CMD = f"""cd scripts && deepspeed train_deepspeed.py \
"""

print(f'Running command: \n{TRAIN_DEEPSPEED_CMD}')
! {TRAIN_DEEPSPEED_CMD}

Running command: 
cd scripts && deepspeed train_deepspeed.py 
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[2024-06-10 02:17:43,891] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
df: ‘/home/ec2-user/.triton/autotune’: No such file or directory
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Infe

<br>

## 3. SageMaker Training 준비
---
SageMaker에 대한 대표적인 오해가 여전히 많은 분들이 SageMaker 훈련을 위해 소스 코드를 전면적으로 수정해야 한다고 생각합니다. 하지만, 실제로는 별도의 소스 코드 수정 없이 기존 여러분이 사용했던 파이썬 스크립트에 SageMaker 훈련에 필요한 SageMaker 전용 환경 변수들만 추가하면 됩니다.

SageMaker 훈련은 훈련 작업을 호출할 때, 
- 1) 훈련 EC2 인스턴스 프로비저닝
- 2) 컨테이너 구동을 위한 도커 이미지 및 훈련 데이터 다운로드
- 3) 컨테이너 구동
- 4) 컨테이너 환경에서 훈련 수행
- 5) 컨테이너 환경에서 S3의 특정 버킷에 저장
- 6) 훈련 인스턴스 종료로 구성됩니다. 따라서, 훈련 수행 로직은 아래 예시와 같이 기존 개발 환경과 동일합니다.
    - 아래와 같이 구동된 컨테이너 안에서 아래의 경로에서 , 아래와 같은 형식으로 훈련 스크립트가 실행 됩니다.

        `/opt/conda/bin/python train_hf.py --num_epochs 5 --train_batch_size 32 ...`

이 과정에서 컨테이너 환경에 필요한 환경 변수(예: 모델 경로, 훈련 데이터 경로) 들은 사전에 지정되어 있으며, 이 환경 변수들이 설정되어 있어야 훈련에 필요한 파일들의 경로를 인식할 수 있습니다. 대표적인 환경 변수들에 대한 자세한 내용은 https://github.com/aws/sagemaker-containers#important-environment-variables 을 참조하세요.


### 로컬 모드 혹은 클라우드 모드 설정
- 아래를 주석으로 조절 하세요.

In [20]:
# USE_LOCAL_MODE = True
USE_LOCAL_MODE = False

if USE_LOCAL_MODE:
    instance_type = 'local_gpu'
    instance_count = 1
    batch_size = 32
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    print("## Local mode is set")

else:
    instance_type = 'ml.g5.2xlarge'
    # instance_type = 'ml.p3.8xlarge'
    # instance_type = 'ml.p3.16xlarge'
    instance_count = 1
    batch_size = 32
    sagemaker_session = sagemaker.session.Session()
    print(f"## Cloud mode is set with {instance_type} and {instance_count} of instance_count")



## Cloud mode is set with ml.g5.2xlarge and 1 of instance_count


## Hyperparameter 설정

In [21]:
# hyperparameters, which are passed into the training job
hyperparameters = {
    'num_epochs': 3,                    # number of training epochs
    'seed': 42,                         # seed
    'train_batch_size': batch_size,     # batch size for training
    'eval_batch_size': batch_size*2,    # batch size for evaluation
    'warmup_steps': 0,                  # warmup steps
    'learning_rate': 3e-5,              # learning rate used during training
    'model_id': model_id                # pre-trained model
}

### mpi_options 설정

In [22]:
if instance_type == 'local_gpu':
    import torch

    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs: {num_gpus}")    

    mpi_options = {
    "enabled" : True,            # Required
    "processes_per_host" : num_gpus,    # Required
    # "custom_mpi_options" : "--mca btl_vader_single_copy_mechanism none"
}
elif instance_type == 'ml.p3.16xlarge' :
    mpi_options = {
        "enabled" : True,            # Required
        "processes_per_host" : 8,    # Required
        # "custom_mpi_options" : "--mca btl_vader_single_copy_mechanism none"
    }    
elif instance_type == 'ml.p3.8xlarge' :
    mpi_options = {
        "enabled" : True,            # Required
        "processes_per_host" : 4,    # Required
        # "custom_mpi_options" : "--mca btl_vader_single_copy_mechanism none"
    }    
    
else:
    mpi_options = {
        "enabled" : True,            # Required
        "processes_per_host" : 1,    # Required
        # "custom_mpi_options" : "--mca btl_vader_single_copy_mechanism none"
    }

mpi_options



{'enabled': True, 'processes_per_host': 1}

### 훈련 도커 이미지 설정

In [23]:
from sagemaker.pytorch import PyTorch
#image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/huggingface-pytorch-training:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04"
image_uri = '763104351884.dkr.ecr.{}.amazonaws.com/pytorch-training:1.12.1-gpu-py38-cu113-ubuntu20.04-sagemaker'.format(region)
print("image_uri: \n", image_uri)


image_uri: 
 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.12.1-gpu-py38-cu113-ubuntu20.04-sagemaker


### Estimator 생성

In [24]:

# define Training Job Name 
job_name = f'deepspeed-nsmc-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
chkpt_s3_path = f's3://{sess.default_bucket()}/{s3_prefix}/native/checkpoints'

# create the Estimator
sm_estimator = PyTorch(
    entry_point           = 'train_deepspeed.py',  # fine-tuning script used in training jon
    source_dir            = './scripts',        # directory where fine-tuning script is stored
      image_uri = image_uri,
    instance_type         = instance_type,      # instances type used for the training job
    instance_count        = instance_count,     # the number of instances used for training
    base_job_name         = job_name,           # the name of the training job
    role                  = role,               # IAM role used in training job to access AWS ressources, e.g. S3
    sagemaker_session=sagemaker_session,
    py_version            = 'py38',             # the python version used in the training job
    hyperparameters       = hyperparameters,    # the hyperparameter used for running the training job
    distribution          = {"mpi": mpi_options},
    disable_profiler     = True,
    debugger_hook_config  = False,
    #keep_alive_period_in_seconds = 20*60     # warm pool    
    #checkpoint_s3_uri     = chkpt_s3_path,
    #checkpoint_local_path ='/opt/ml/checkpoints',  
)

### 훈련에 사용할 데이터 설정
- 위에서 업로드한 S3 의 경로를 기술 함.

In [25]:
# define a data input dictonary with our uploaded s3 uris
data = {
    'train': train_input_path,
    'eval': eval_input_path
}

## 4. 훈련 실행

In [26]:
# starting the train job with our uploaded datasets as input
sm_estimator.fit(data, wait=False)
train_job_name = sm_estimator.latest_training_job.job_name

[{session.py:1002} INFO - Creating training-job with name: deepspeed-nsmc-2024-06-10-03-41-59-2024-06-10-03-42-01-988


### 훈련 잡과 클라우드 워치 로그 경로 생성

In [27]:
from IPython.display import display, HTML

def make_console_link(region, train_job_name, train_task='[Training]'):
    train_job_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{train_job_name}">Training Job</a></b>'   
    cloudwatch_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={region}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={train_job_name};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a></b>'
    return train_job_link, cloudwatch_link  
        
train_job_link, cloudwatch_link = make_console_link(region, train_job_name, '[PyTorch DeepSpeed Training]')

display(HTML(train_job_link))
display(HTML(cloudwatch_link))

### 훈련 로그 보기

In [28]:
sm_estimator.logs()

2024-06-10 03:42:02 Starting - Starting the training job
2024-06-10 03:42:02 Pending - Training job waiting for capacity......
2024-06-10 03:42:48 Pending - Preparing the instances for training...
2024-06-10 03:43:34 Downloading - Downloading the training image.....................
2024-06-10 03:46:50 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-06-10 03:47:16,307 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-06-10 03:47:16,327 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-10 03:47:16,338 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-06-10 03:47:16,340 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-06-10 03:47:16,562 sagemaker-training-toolkit INFO     Installing dep

<br>

## 5. (Optional) Inference
---

### Copy S3 model artifact to local directory
- S3에 저장된 모델 아티팩트를 로컬 경로로 복사하여 압축을 해제합니다.

In [29]:
import json, os

local_model_dir = 'model_from_sagemaker'

if not os.path.exists(local_model_dir):
    os.makedirs(local_model_dir)

!aws s3 cp {sm_estimator.model_data} {local_model_dir}/model.tar.gz
!tar -xzf {local_model_dir}/model.tar.gz -C {local_model_dir}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
download: s3://sagemaker-us-east-1-057716757052/deepspeed-nsmc-2024-06-10-03-41-59-2024-06-10-03-42-01-988/output/model.tar.gz to model_from_sagemaker/model.tar.gz
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'


### Base 모델 및 토큰나이저 로딩

In [30]:
import torch
import transformers
import numpy as np
from collections import OrderedDict
from transformers import BertForSequenceClassification, AutoTokenizer
        
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)#.to(device)

/home/ec2-user/anaconda3/envs/deepspeed-py310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializi

In [31]:
## model

### Load DDP model to a non-DDP model
- 데이터 병렬화를 적용하여 모델을 훈련하면 모델의 weight의 key값에 `module`이 붙게 되어 모델 로딩 시 오류가 발생합니다. 따라서, 이를 제거해 주는 후처리 과정이 필요합니다. - 후처리가 번거롭다면, DDP로 훈련 후 저장할 때 명시적으로 `module`를 제외하고 저장하는 방법도 있습니다.
    - 참조: https://discuss.pytorch.org/t/how-to-switch-model-trained-on-2-gpus-to-1-gpu/20039

    ```python
    model_to_save = model.module if hasattr(model, 'module') else model
    ...
    model_to_save.state_dict()
    torch.save({'model': model_to_save.state_dict())
    ```


In [32]:
import glob
model_filename = glob.glob(f'{local_model_dir}/*.pt')[0]
print("model_filename: ", model_filename)
state_dict = torch.load(model_filename)
for idx, key in enumerate(state_dict):
    print(key)
    if idx == 25:
        break

model_filename:  model_from_sagemaker/model.pt
module.bert.embeddings.position_ids
module.bert.embeddings.word_embeddings.weight
module.bert.embeddings.position_embeddings.weight
module.bert.embeddings.token_type_embeddings.weight
module.bert.embeddings.LayerNorm.weight
module.bert.embeddings.LayerNorm.bias
module.bert.encoder.layer.0.attention.self.query.weight
module.bert.encoder.layer.0.attention.self.query.bias
module.bert.encoder.layer.0.attention.self.key.weight
module.bert.encoder.layer.0.attention.self.key.bias
module.bert.encoder.layer.0.attention.self.value.weight
module.bert.encoder.layer.0.attention.self.value.bias
module.bert.encoder.layer.0.attention.output.dense.weight
module.bert.encoder.layer.0.attention.output.dense.bias
module.bert.encoder.layer.0.attention.output.LayerNorm.weight
module.bert.encoder.layer.0.attention.output.LayerNorm.bias
module.bert.encoder.layer.0.intermediate.dense.weight
module.bert.encoder.layer.0.intermediate.dense.bias
module.bert.encoder.lay

### 훈련된 모델 파라미터 로딩

In [33]:
new_state_dict = {}
for key in state_dict:
    new_key = key.replace('module.','')
    new_state_dict[new_key] = state_dict[key]

model.load_state_dict(new_state_dict)

<All keys matched successfully>

### 모델 추론

In [34]:
def inference_model(model, tokenizer, text):
    encode_plus_token = tokenizer.encode_plus(
        text,
        max_length=128,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding="max_length",
        return_attention_mask=True,
        return_tensors="pt",
        truncation=True
    )

    output = model(**encode_plus_token)
    softmax_fn = torch.nn.Softmax(dim=1)
    softmax_output = softmax_fn(output[0])
    _, prediction = torch.max(softmax_output, dim=1)

    predicted_class_idx = prediction.item()
    score = softmax_output[0][predicted_class_idx]
    print(f"predicted_class: {predicted_class_idx}, score={score}")



In [35]:
text = "이 영화 너무 재미있어요"
inference_model(model, tokenizer, text)

predicted_class: 1, score=0.9877290725708008


In [36]:
text = "'기생충'은 많은 사람들로부터 호평을 받았고, 매우 사실적으로 묘사 했습니다."
inference_model(model, tokenizer, text)

predicted_class: 1, score=0.9625192284584045
